In [1]:
from parse import proof, ProofVisitor
from proof import Proof

In [2]:
from test import Test

In [4]:
from formula import BTrue
type(BTrue())

formula.BTrue

In [3]:
tree = proof.parse("""
1. ! [X0] : p(X0,f(X0)) [input]
2. ! [X0,X1] : (p(X0,X1) => q(X0,g(X1))) [input]
3. ! [X0,X1] : (q(X0,X1) => r(X0,h(X1))) [input]
4. ! [X0] : ? [X1] : r(X0,X1) [input]
5. ~! [X0] : ? [X1] : r(X0,X1) [negated conjecture 4]
6. ! [X0,X1] : (q(X0,g(X1)) | ~p(X0,X1)) [ennf transformation 2]
7. ! [X0,X1] : (r(X0,h(X1)) | ~q(X0,X1)) [ennf transformation 3]
8. ? [X0] : ! [X1] : ~r(X0,X1) [ennf transformation 5]
9. ? [X0] : ! [X1] : ~r(X0,X1) => ! [X1] : ~r(sK0,X1) [choice axiom]
10. ! [X1] : ~r(sK0,X1) [skolemisation 8,9]
11. p(X0,f(X0)) [cnf transformation 1]
12. q(X0,g(X1)) | ~p(X0,X1) [cnf transformation 6]
13. r(X0,h(X1)) | ~q(X0,X1) [cnf transformation 7]
14. ~r(sK0,X1) [cnf transformation 10]
15. ~q(sK0,X0) [resolution 13,14]
16. ~p(sK0,X0) [resolution 15,12]
17. $false [resolution 16,11]
"""
)
pv = ProofVisitor()
pv.visit(tree)
p = Proof.parse(pv.result)

In [4]:
p

Resolution(arguments=[Resolution(arguments=[Resolution(arguments=[CnfTransformation(arguments=[EnnfTransformation(arguments=[Input(arguments=[], mgu=None, head=Forall(bound_variables=[Variable(name='X0'), Variable(name='X1')], arguments=[Implication(arguments=[Atom(name='q', arguments=[Variable(name='X0'), Variable(name='X1')]), Atom(name='r', arguments=[Variable(name='X0'), FunctionApplication(name='h', arguments=[Variable(name='X1')])])])]))], mgu=None, head=Forall(bound_variables=[Variable(name='X0'), Variable(name='X1')], arguments=[Disjunction(arguments=[Atom(name='r', arguments=[Variable(name='X0'), FunctionApplication(name='h', arguments=[Variable(name='X1')])]), Negation(arguments=[Atom(name='q', arguments=[Variable(name='X0'), Variable(name='X1')])])])]))], mgu=None, head=Disjunction(arguments=[Atom(name='r', arguments=[Variable(name='X0'), FunctionApplication(name='h', arguments=[Variable(name='X1')])]), Negation(arguments=[Atom(name='q', arguments=[Variable(name='X0'), Varia

In [5]:
p.head

BFalse(arguments=[])

In [6]:
p.arguments[0].head

Negation(arguments=[Atom(name='p', arguments=[Constant(name='sK0'), Variable(name='X0')])])

In [7]:
print(p.arguments[0].head)

~(p(sK0,X0))


In [8]:
p.arguments[0].head.variables()

{Variable(name='X0')}

In [9]:
print(p)

1. False [Resolution 2 15]{'X0 -> f(sK0)'}{'X0 -> sK0'}
2. ~(p(sK0,X0)) [Resolution 3 12]{'X0 -> g(X1)'}{'X0 -> sK0'}
3. ~(q(sK0,X0)) [Resolution 4 7]{'X0 -> sK0'}{'X1 -> h(X1)'}
4. (r(X0,h(X1))) | (~(q(X0,X1))) [CnfTransformation 5]
5. ! [X0,X1]: (r(X0,h(X1))) | (~(q(X0,X1))) [EnnfTransformation 6]
6. ! [X0,X1]: (q(X0,X1)) => (r(X0,h(X1))) [Input]
7. ~(r(sK0,X1)) [CnfTransformation 8]
8. ! [X1]: ~(r(sK0,X1)) [Skoemisation 9 11]
9. ? [X0]: ! [X1]: ~(r(X0,X1)) [EnnfTransformation 10]
10. ~(! [X0]: ? [X1]: r(X0,X1)) [NegatedConjecture]
11. (? [X0]: ! [X1]: ~(r(X0,X1))) => (! [X1]: ~(r(sK0,X1))) [ChoiceAxiom]
12. (q(X0,g(X1))) | (~(p(X0,X1))) [CnfTransformation 13]
13. ! [X0,X1]: (q(X0,g(X1))) | (~(p(X0,X1))) [EnnfTransformation 14]
14. ! [X0,X1]: (p(X0,X1)) => (q(X0,g(X1))) [Input]
15. p(X0,f(X0)) [CnfTransformation 16]
16. ! [X0]: p(X0,f(X0)) [Input]



In [10]:
p = p.make_unique_variables()
print(p)

1. False [Resolution 2 15]{'X110 -> f(sK0)'}{'X01 -> sK0'}
2. ~(p(sK0,X110)) [Resolution 3 12]{'X1000 -> g(X110)'}{'X010 -> sK0'}
3. ~(q(sK0,X1000)) [Resolution 4 7]{'X0000 -> sK0'}{'X1100 -> h(X1000)'}
4. (r(X0000,h(X1000))) | (~(q(X0000,X1000))) [CnfTransformation 5]
5. ! [X0,X1]: (r(X0,h(X1))) | (~(q(X0,X1))) [EnnfTransformation 6]
6. ! [X0,X1]: (q(X0,X1)) => (r(X0,h(X1))) [Input]
7. ~(r(sK0,X1100)) [CnfTransformation 8]
8. ! [X1]: ~(r(sK0,X1)) [Skoemisation 9 11]
9. ? [X0]: ! [X1]: ~(r(X0,X1)) [EnnfTransformation 10]
10. ~(! [X0]: ? [X1]: r(X0,X1)) [NegatedConjecture]
11. (? [X0]: ! [X1]: ~(r(X0,X1))) => (! [X1]: ~(r(sK0,X1))) [ChoiceAxiom]
12. (q(X010,g(X110))) | (~(p(X010,X110))) [CnfTransformation 13]
13. ! [X0,X1]: (q(X0,g(X1))) | (~(p(X0,X1))) [EnnfTransformation 14]
14. ! [X0,X1]: (p(X0,X1)) => (q(X0,g(X1))) [Input]
15. p(X01,f(X01)) [CnfTransformation 16]
16. ! [X0]: p(X0,f(X0)) [Input]



In [11]:
print(p.step1())

1. False [Resolution 2 15]{'X110 -> f(sK0)'}{'X01 -> sK0'}
2. ~(p(sK0,f(sK0))) [Resolution 3 12]{'X1000 -> g(X110)'}{'X010 -> sK0'}
3. ~(q(sK0,g(f(sK0)))) [Resolution 4 7]{'X0000 -> sK0'}{'X1100 -> h(X1000)'}
4. (r(sK0,h(g(f(sK0))))) | (~(q(sK0,g(f(sK0))))) [CnfTransformation 5]
5. ! [X0,X1]: (r(X0,h(X1))) | (~(q(X0,X1))) [EnnfTransformation 6]
6. ! [X0,X1]: (q(X0,X1)) => (r(X0,h(X1))) [Input]
7. ~(r(sK0,h(g(f(sK0))))) [CnfTransformation 8]
8. ! [X1]: ~(r(sK0,X1)) [Skoemisation 9 11]
9. ? [X0]: ! [X1]: ~(r(X0,X1)) [EnnfTransformation 10]
10. ~(! [X0]: ? [X1]: r(X0,X1)) [NegatedConjecture]
11. (? [X0]: ! [X1]: ~(r(X0,X1))) => (! [X1]: ~(r(sK0,X1))) [ChoiceAxiom]
12. (q(sK0,g(f(sK0)))) | (~(p(sK0,f(sK0)))) [CnfTransformation 13]
13. ! [X0,X1]: (q(X0,g(X1))) | (~(p(X0,X1))) [EnnfTransformation 14]
14. ! [X0,X1]: (p(X0,X1)) => (q(X0,g(X1))) [Input]
15. p(sK0,f(sK0)) [CnfTransformation 16]
16. ! [X0]: p(X0,f(X0)) [Input]



In [14]:
print(p.step1().step2())

1. r(sK0,h(g(f(sK0)))) [Resolution 2 11]{'X110 -> f(sK0)'}{'X01 -> sK0'}
2. (~(p(sK0,f(sK0)))) | (r(sK0,h(g(f(sK0))))) [Resolution 3 8]{'X1000 -> g(X110)'}{'X010 -> sK0'}
3. (~(q(sK0,g(f(sK0))))) | (r(sK0,h(g(f(sK0))))) [Resolution 4 7]{'X0000 -> sK0'}{'X1100 -> h(X1000)'}
4. (r(sK0,h(g(f(sK0))))) | (~(q(sK0,g(f(sK0))))) [CnfTransformation 5]
5. ! [X0,X1]: (r(X0,h(X1))) | (~(q(X0,X1))) [EnnfTransformation 6]
6. ! [X0,X1]: (q(X0,X1)) => (r(X0,h(X1))) [Input]
7. (~(r(sK0,h(g(f(sK0)))))) | (r(sK0,h(g(f(sK0))))) [Input]
8. (q(sK0,g(f(sK0)))) | (~(p(sK0,f(sK0)))) [CnfTransformation 9]
9. ! [X0,X1]: (q(X0,g(X1))) | (~(p(X0,X1))) [EnnfTransformation 10]
10. ! [X0,X1]: (p(X0,X1)) => (q(X0,g(X1))) [Input]
11. p(sK0,f(sK0)) [CnfTransformation 12]
12. ! [X0]: p(X0,f(X0)) [Input]

